# Von Frey XO sequence calculator and plotter

### This R script is developend in Andrei V. Chernov's Lab at the University of California San Diego.

#### Version 0.1. https://github.com/chernov-lab/VonFreyTest

#### The script is based on experimental work in Drs. T.L. Yaksch (UCSD) and V.I. Shubayev (UCSD).

##### Parametric data were derived from: 
S R Chaplan  1 , F W Bach, J W Pogrel, J M Chung, T L Yaksh
Quantitative assessment of tactile allodynia in the rat paw
J Neurosci Methods, 1994 Jul;53(1):55-63. doi: 10.1016/0165-0270(94)90144-9.


## Provide file name of the excel file that includes **data** and **meta** tabs
### The **data** tab stores observation and last filament results
### The **meta** tab establishes comparison pairs

In [ ]:
data_file <- "test_data.xlsx"  # <<<- provide excel file name with VF data. File should be uploaded in the main folder

## Set up test parameters:

In [ ]:
paw <- 'LEFT' #'RIGHT',  # define paw to use
sex <- "female"  # define sex of animals if needed. Assign "" is sex is not defined
species <- "rat" # define species of animal rat or mouse
res_file <- paste('result', data_file, sep='.')  # result file
post_hoc_method <- "bonferroni" # Allowed values include "holm", "hochberg", "hommel", "bonferroni", "BH", "BY", "fdr", "none". If you don't want to adjust the p value (not recommended), use p.adjust.method = "none".
label_id <- F # set TRUE if want to label animal IDs below threshold

In [ ]:
library(readr)
library(ggplot2)
library(ggpubr)
library(ggiraphExtra)
library(dplyr)
library(tidyverse)
library(rstatix)
library(DescTools)
library(readxl)
library(writexl)
library(lubridate)
library(emmeans)
library(ggpubr)

### Define some custom functions

In [ ]:
`%notin%` <- Negate(`%in%`)

acPage = function(w1=12, h1=6, r1=1, c1=1, m1=2, m2=2, m3=2, m4=2){
  options(repr.plot.width=w1, repr.plot.height= h1) 
  par(mfrow=c(r1,c1), mar=c(m1,m2,m3,m4))
}

### Set on-screen plotting parameters

In [ ]:
acPage()

In [ ]:
# Folder definitions
main_folder = ""
data_folder <- "files"
par_folder <- "parameters"

sex <- toupper(sex)

if (species == "rat"){
    pain_threshold <- 5
    max_res <- 15
}

### Read species specific parameters 
### Perform Von Frey Test calculations

In [ ]:
XO_file <- paste(species, "final.previous.filaments.XO.csv", sep='.')
stat_file <- paste(species, "observation-statistics.csv", sep='.')

observation_table <- read.csv(file.path(par_folder, stat_file), row.names=1, stringsAsFactors=FALSE)
XO.table <- read.csv(file.path(par_folder, XO_file), row.names=1, stringsAsFactors=FALSE) #, col_types = "nnn")

VonFrey = function(obs='', last=0){
    obs <- toupper(obs)
    res <- 0
    if (obs == '' | last == 0) { return (-1)} 
    else if (obs == 'OOOOO' & last == 5.18) { res <- max_res } 
    else if (obs == 'XXXX' & last == 3.61) { res <- 0.2 }
    else if (length(which(rownames(XO.table) == last)) == 0 | length(which(rownames(observation_table) == obs))== 0) { return (-1)}
    else {
    a <- substr(obs, nchar(obs)-1, nchar(obs)-1)
    prev <- XO.table[which(rownames(XO.table) == last), a]
    dif <- abs(last - prev)
    p50 <- last + dif * observation_table[which(rownames(observation_table) == obs),'STATISTIC']
    res <- (10**p50)/10000
    if (res > 15) { res <- max_res }
    if (res < 0.2) { res <- 0.20 }
    return(as.numeric(sprintf("%.2f", res)))
    }
}
TruncateObservation = function(obs='', met=2){
    obs <- toupper(obs)
    if (obs == '') { return ("") } 
    if (met == 2) { return (obs) }
    if (met == 1) {  # Jenny's short method
    len <- nchar(obs)  
    Xpos <- StrPos(obs, 'X') 
    if (is.na(Xpos)) { return (obs) }   
    obs1 <- substr(obs, Xpos, len)
 
    return (obs1)
    }
}

## Read DATA and META files from main folder

In [ ]:
x <- readxl::read_excel(data_file, sheet = "data") 
meta <- readxl::read_excel(data_file, sheet = "meta")

In [ ]:
x <- x %>% rename_with(toupper)
meta <- meta %>% rename_with(toupper)

# convert key parameters to uppercase
x <- x %>% dplyr::mutate (
                DATE = ymd(DATE),
                SEX = toupper(SEX), 
                OBSERVATION = toupper(OBSERVATION),
                PAW = toupper(PAW))
x <- x %>% dplyr::arrange(SEX, TIMEPOINT, PAW)
head(x)
head(meta)

### filter by specific sex if **sex** is assigned

In [ ]:
if (sex != ""){ x <- x %>% filter (SEX == sex) }

### Calculate **Von Frey Test** values

In [ ]:
x <- x %>% dplyr::mutate(RESULT = mapply(VonFrey, OBSERVATION, LAST))

In [ ]:
head(x, 5)

### Check for errors reported by Von Frey calculator
#### Negative RESULTs (-1) indicate an error most likely related to incorrect XO data entry

In [ ]:
x %>% filter(RESULT < 0)

### Prepare dataset for ANOVA analysis and plotting

In [ ]:
x <- x %>% filter(RESULT > 0)
lp_x_names <- c('SEX', 'GROUP', 'PAW', 'TIMEPOINT', 'RESULT', 'ANIMAL_ID')
lp_meta_names <- c('COMPARISON', 'TREATMENT', 'REFERENCE')

In [ ]:
lp <- x %>% 
    select(all_of(lp_x_names)) %>% 
        mutate(DAY = TIMEPOINT, 
               TIMEPOINT = as.integer(TIMEPOINT),
               DAY = factor(TIMEPOINT),
               GROUP = factor(GROUP),
               ID = row_number()) %>% 
        print

In [ ]:
print(meta)

### Set up comparison pairs

In [ ]:
meta <- meta %>% mutate(COMPARISON = mapply(sprintf, "%s-%s", TREATMENT, REFERENCE)) %>% print

In [ ]:
comparisons <- meta %>% select (TREATMENT, REFERENCE) %>% t %>% as_tibble %>% as.list %>% print

### Filter by LEFT or RIGHT paw

In [ ]:
if (paw != "") { lp <- lp %>% filter(PAW == paw) }

In [ ]:
min_res <- lp %>% select (RESULT) %>% min %>% print
max_res <- lp %>% select (RESULT) %>% max %>% print

## Summary statistics
Compute some summary statistics (count, mean and sd) of the variable weight organized by groups:

# Two-way ANOVA

## Summary statistics
Compute the mean and the SD (standard deviation) of the score by groups:

In [ ]:
lp_stat <- lp %>%
  group_by(GROUP, DAY) %>%
        get_summary_stats(RESULT, type = "mean_se") %>% 
        mutate (TIMEPOINT = as.numeric(strtoi(DAY))) %>% 
        print

## Visualization
Create a box plot of the score by gender levels, colored by education levels:

In [ ]:
acPage(w1=15, h1=6)
bxp <- lp %>% ggboxplot(
    x = "GROUP", y = "RESULT",
    color = "GROUP", 
    palette = "jco",
    ) +
    rremove("x.text") +
    geom_hline(yintercept=5, linetype="dashed", color = "orange", size=1) +
    facet_grid( ~ TIMEPOINT, labeller = "label_both")

plot(bxp)

In [ ]:
lp_outs <- lp %>%
  group_by(GROUP, DAY) %>%
    identify_outliers(RESULT) %>% 
        filter(is.extreme == TRUE) %>% print()

lp <- lp %>% anti_join(lp_outs, by = "ID") %>% print()

### Build the linear model

In [ ]:
model  <- lm(RESULT ~ GROUP * DAY, data = lp)

### Create a QQ plot of residuals

In [ ]:
model %>% residuals %>% ggqqplot

### Compute Shapiro-Wilk test of normality

In [ ]:
model %>% residuals %>% shapiro_test

In [ ]:
lp %>% group_by(GROUP) %>% shapiro_test(RESULT) %>% print

In [ ]:
ggqqplot(lp, "RESULT", ggtheme = theme_bw()) +
  facet_grid( GROUP ~ TIMEPOINT, labeller = "label_both")

### Calculate ANOVA statistics table with post hoc corrections 

In [ ]:
pwc <- lp %>% 
  group_by(DAY) %>%
      emmeans_test(RESULT ~ GROUP, model = model, 
            comparisons = comparisons,
                   p.adjust.method = post_hoc_method) 

### List most significant differences between groups

In [ ]:
pwc %>% filter(p.adj < 0.1) %>% arrange (p.adj) %>% print

### Add formatted P-value for graphics

In [ ]:
pwc$p.format <- p_format(pwc$p, accuracy = 0.0001, leading.zero = FALSE)
head(pwc)

## Plot all data on one graph

In [ ]:
acPage(w1=15, h1=10)

x_max <- max(lp$TIMEPOINT) + 1
days <- order(unique(lp$TIMEPOINT), decreasing=F)

p <- ggline(lp, 
            x = "TIMEPOINT",
            y = "RESULT", 
            size = 1.5,            
            color = "GROUP",
            shape = "GROUP",
            point.size = 5,
            linetype = "GROUP",
            ylim = c(0, max_res),
            add = c("mean_se", "jitter" ),
            add.params = list(width = 0.1, shape = 15),
            palette = "GROUP"
            )  +
    geom_point(size = 3, aes(color = GROUP, shape = GROUP)) +
    xlab("Time after IS injection, days") +
    ylab("Tactile threshold, g") +
    ggtitle( sprintf( "von Frey behavior tests in %s paws", tolower (paw))) +
    theme(text = element_text(size=20, color = "black", angle = 0, hjust = .5, vjust = 0, face = "italic"),
    axis.text.x = element_text(hjust=1)) +
    geom_hline(yintercept=5, linetype="dashed", color = "orange", size=1.2) +
    scale_y_continuous(breaks= 0 : round(max_res) * max_res/3, expand = expansion(mult = c(0, 0.1))  ) +
    scale_x_continuous(breaks= 0 : x_max * 1) 
plot(p)

In [ ]:
### Plot this graph into a PDF file

In [ ]:
pdf(sprintf("vonFrey plot.pdf"), width = 15, height = 10)
plot(p)
dev.off()

### Ensure X axis coordinates correctly mapped

In [ ]:
pwc <- pwc %>% mutate(x = as.numeric(DAY), xmin = as.numeric(strtoi(DAY)), xmax = as.numeric(strtoi(DAY)) + 0.32) %>% arrange(x)

## Plot graphs for individual comparisons and save in PDF files

In [ ]:
acPage(w1=12, h1=7)

for (i in 1:nrow(meta)){

lp_plot <- lp %>% filter ( GROUP == meta$TREATMENT[i] | GROUP == meta$REFERENCE[i] ) 
pwc.f <- pwc %>% filter(group1 == meta$TREATMENT[i] & group2 == meta$REFERENCE[i])
    
pp <- ggline(
            lp_plot, 
            y = "RESULT", 
            x = "TIMEPOINT", 
            color = "GROUP", 
            ylim = c(0, max_res + 1),
            linetype = "GROUP",
            size = 1.5, binwidth=0.1, 
            add = c("mean_se"),
            palette = c("red3", "blue1"),
            ) +  scale_x_continuous(breaks = 0:21*1) +
    stat_pvalue_manual( 
            pwc.f, 
            label = "p.adj.signif",
            position = position_dodge(0.6), 
            remove.bracket = T,
            size = 11,
            y.position = 15.5,
            hide.ns = T,
            color = "black"
            ) +
    geom_point(aes(shape = GROUP, color = GROUP, size = 10), alpha = 5/10, show.legend = F) +
    xlab(label = "Time after IS injection, days") +
    ylab(label = "Tactile threshold, g") +
    ggtitle( sprintf( "%s vs %s IS by von Frey tests in %s %s paws. Mean\u00B1SE, %s post-hoc", 
                     meta$TREATMENT[i], meta$REFERENCE[i], species, tolower (paw), str_to_title(post_hoc_method))) +
    theme_classic() +
    geom_hline(yintercept=pain_threshold, linetype="dashed", color = "orange", size=1.2) +
    theme(text = element_text(size=16, color = "black", angle = 0, hjust = .5, vjust = 0, face = "italic")) +
    geom_text(aes(label=ifelse(label_id & RESULT<=pain_threshold,as.character(ANIMAL_ID),'')),hjust = 0, vjust = 1.5, size = 4)

    plot(pp)

    pdf(sprintf("vonFrey plot %s vs %s in %s %s paw.pdf", meta$TREATMENT[i], meta$REFERENCE[i], species, tolower (paw)), width = 12, height = 7)
        plot(pp)
    dev.off()
}

In [ ]:
for (i in 1:nrow(meta)){
sprintf("Significance scores for %s vs %s, post-hoc %s", meta$TREATMENT[i], meta$REFERENCE[i], post_hoc_method) %>% print
pwc %>% filter(group1 == meta$TREATMENT[i] & group2 == meta$REFERENCE[i]) %>% 
    mutate (DAY = as.integer(DAY)) %>% 
    arrange (DAY) %>% 
    select (DAY, group1, group2, p, p.adj, p.adj.signif) %>%
    print
}
pwc <- pwc %>% arrange (group1, group2, DAY) %>% print

### Write results in file with **Results.** prefix

In [ ]:
write_xlsx(list(data = x, meta = meta, stats = pwc), res_file)

In [ ]:
sessionInfo()